In [ ]:
import os.path as osp
import sys
import random

path = osp.dirname(osp.abspath(''))
sys.path.append(path)
sys.path.append(osp.join(path, "./../open_biomed"))
path

In [ ]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [ ]:

def read_text(path=None):
    with open(path, 'r') as f:

        text_list = []
        for index, line in enumerate(f):
            text = line.strip()
            text_list.append(text)
    return text_list
path='./extracted_chemistry_keywords.txt'
extracted_text=read_text(path=path)
path='./text_after_structure_filter.txt'
texts=read_text(path=path)

path='./smis_after_structure_filter.txt'
smis=read_text(path=path)

In [ ]:
len(extracted_text),len(texts),len(smis)

In [ ]:
# 线性核函数，用 PyTorch 实现
def linear_kernel(X):
    return torch.mm(X, X.T)  # 使用矩阵乘法

# 居中核矩阵
def center_kernel(K):
    n = K.shape[0]
    H = torch.eye(n, device=K.device) - torch.ones((n, n), device=K.device) / n  # 中心化矩阵 H
    return torch.mm(H, torch.mm(K, H))

# CKA 实现，计算两个张量 X 和 Y 的 CKA
def CKA(X, Y):
    # 计算线性核矩阵
    K = linear_kernel(X)
    L = linear_kernel(Y)

    # 居中核矩阵
    Kc = center_kernel(K)
    Lc = center_kernel(L)

    # 计算 HSIC 值
    hsic_XY = torch.trace(torch.mm(Kc, Lc))
    hsic_XX = torch.trace(torch.mm(Kc, Kc))
    hsic_YY = torch.trace(torch.mm(Lc, Lc))

    # 计算 CKA 值
    cka_value = hsic_XY / torch.sqrt(hsic_XX * hsic_YY)
    return cka_value

In [ ]:
from metrics import cka,gram_linear
# structure_feats_np = structure_feats[indes].cpu().detach().numpy()
# text_feats_np = text_feats[indes].cpu().detach().numpy()
# # Calculate Gram matrices for structure_feats and text_feats
def de_biased_cka(structure_feats,text_feats):
    structure_feats_np = structure_feats.cpu().detach().numpy()
    text_feats_np = text_feats.cpu().detach().numpy()
    gram_structure = gram_linear(structure_feats_np)
    gram_text = gram_linear(text_feats_np)

    # Calculate CKA between the two Gram matrices
    cka_value = cka(gram_structure, gram_text, debiased=True)

    return cka_value

In [ ]:
import torch.nn.functional as F

def retrieve(A, B, K=1):
    """
    检索任务函数，输入两个 tensor A 和 B，返回 A 和 B 匹配的索引值、匹配的分数和正确匹配的条目数。

    参数：
    - A: torch.Tensor，待匹配的 query tensor，形状 (n, d)。
    - B: torch.Tensor，待匹配的 target tensor，形状 (n, d)。
    - K: int，检索 top K 匹配。

    返回：
    - indices: 每个 A 对应的最相似 B 的索引值，形状 (n, K)。
    - scores: 每个 A 对应的最相似 B 的分数，形状 (n, K)。
    - correct_total: 匹配正确的总条目数。
    """

    # 计算 A 和 B 之间的余弦相似度 (n, n)
    similarities = F.cosine_similarity(A.unsqueeze(1), B.unsqueeze(0), dim=-1)
    # 获取相似度最高的前 K 个条目的索引和分数
    topk_scores, topk_indices = torch.topk(similarities, K, dim=1)
    # 计算总的正确匹配数量
    # 正确的定义：A[i] 和 B[i] 相匹配，意味着第 i 个 A 应该匹配第 i 个 B
    correct_total = 0
    n = A.size(0)
    for i in range(n):
        # 检查 top K 的索引是否包含正确匹配
        if i in topk_indices[i]:
            correct_total += 1
    return topk_indices, topk_scores, correct_total

In [ ]:
def overleap(indexA,indexB ):
    C=[]
    D=[]
    for a, b in zip(indexA, indexB):
        overlap = list(set(a) & set(b))
        C.append(overlap)
        D.append([len(overlap)])

    # Print results
    print("Overlap C:", C)
    print("Number of overlaps D:", D)

    sum=0
    for i in C:
        sum=sum+len(i)
    print(sum)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

def KM(embedding=None,num_clusters=5):
    num_clusters = num_clusters  # You can set this based on your requirement

    # Initialize and fit the KMeans model
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(embedding)

    return kmeans

In [ ]:
from sklearn.neighbors import NearestNeighbors

def NN(embedding=None,num=10):
    # Initialize the Nearest Neighbors model
    num_neighbors = num  # Number of nearest neighbors to find
    nn_model = NearestNeighbors(n_neighbors=num_neighbors)

    # Fit the model to your data
    try:
        nn_model.fit(embedding.cpu().numpy())
    except:
        nn_model.fit(embedding.detach().cpu().numpy())

    return nn_model

In [ ]:
len(extracted_text),len(smis),len(texts)

## MolCLR

In [ ]:
# gcn_embeddings=torch.load('gcn_embeddings.pt')
# gcn_out_embeddings=torch.load('gcn_out_embeddings.pt')
# gin_embeddings=torch.load('gin_embeddings.pt')
# gin_out_embeddings=torch.load('gin_out_embeddings.pt')

In [ ]:
# gcn_embeddings.shape,gcn_out_embeddings.shape,gin_embeddings.shape,gin_out_embeddings.shape,

In [ ]:
import torch
from torch_geometric.data import Data
from rdkit import Chem
from rdkit.Chem import AllChem
from MolCLR.models.ginet_molclr import GINet
from MolCLR.models.gcn_molclr import GCN

# Convert a SMILES string into a graph representation
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    Chem.SanitizeMol(mol)
    
    # Extract atom features and edge features
    atom_features = torch.tensor([[atom.GetAtomicNum(), atom.GetChiralTag()] for atom in mol.GetAtoms()], dtype=torch.long)
    bond_features = torch.tensor([[bond.GetBondTypeAsDouble(), bond.GetStereo()] for bond in mol.GetBonds()], dtype=torch.long)
    
    # Build edge_index (source, target) pairs for the molecular graph
    edge_index = torch.tensor([[bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()] for bond in mol.GetBonds()], dtype=torch.long).t().contiguous()
    
    # Create a batch (for a single molecule, the batch index will be all 0s)
    data = Data(x=atom_features, edge_index=edge_index, edge_attr=bond_features, batch=torch.tensor([0] * atom_features.size(0)))
    return data

# Load the GINet model based on the configuration from config.yaml
def load_model(path='',type='gin'):
    # Initialize the GINet model with the exact settings from config.yaml
    if type=='gin':
        model = GINet(num_layer=5, emb_dim=300, feat_dim=512, drop_ratio=0, pool='mean')
    elif type=='gcn':
        model=GCN(num_layer=5, emb_dim=300, feat_dim=512, drop_ratio=0, pool='mean')
    
    # Optionally load pre-trained weights from the "pretrained_gin" model
    pretrained_model_path = path  # Ensure this path exists and is correct
    try:
        model.load_state_dict(torch.load(pretrained_model_path,map_location='cuda:0'))
        print("Loaded pre-trained model from:", pretrained_model_path)
    except FileNotFoundError:
        print("Pre-trained weights not found, training from scratch.")

    model.eval()  # Set model to evaluation mode
    return model


gin_model = load_model(path='./../MolCLR/ckpt/pretrained_gzin/checkpoints/model.pth',type='gin')
gcn_model = load_model(path='./../MolCLR/ckpt/pretrained_gcn/checkpoints/model.pth',type='gcn')


In [ ]:
gcn_h_list=[]
gcn_out_list=[]
gin_h_list=[]
gin_out_list=[]
text_list=[]
et_list=[]
pass_list=[]
compute_list=[]
for index,i in enumerate(smis) :
    # print(index)
    # print(i)
    graph_data = smiles_to_graph(i)
    # print(graph_data)
    try:
        gin_h, gin_out = gin_model(graph_data)
        gin_h_list.append(gin_h)
        gin_out_list.append(gin_out)
        gcn_h, gcn_out = gcn_model(graph_data)
        gcn_h_list.append(gcn_h)
        gcn_out_list.append(gcn_out)
        text_list.append(texts[index])
        et_list.append(extracted_text[index])
        compute_list.append(index)
    except IndexError:
        # print("PASS:",index)
        pass_list.append(index)

In [ ]:
len(compute_list),len(pass_list),len(gin_h_list),len(gin_out_list),len(gcn_h_list),len(gcn_out_list),len(text_list),len(et_list)

In [ ]:
# with open('compute_list.txt','w') as f:
#     for i in compute_list:
#         f.writelines(str(i)+'\n')

In [ ]:
# with open('pass_list.txt','w') as f:
#     for i in pass_list:
#         f.writelines(str(i)+'\n')

In [ ]:
gcn_embeddings = torch.cat(gcn_h_list, dim=0)
gcn_out_embeddings= torch.cat(gcn_out_list, dim=0)
gin_embeddings = torch.cat(gin_h_list, dim=0)
gin_out_embeddings= torch.cat(gin_out_list, dim=0)
gcn_embeddings.shape,gin_embeddings.shape,gcn_out_embeddings.shape,gin_out_embeddings.shape

## SCI-BERT

In [ ]:
from transformers import AutoTokenizer,AutoModel

def SCIBERT(texts=texts):

    tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
    model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')
    model = model.to(device)

    # 将句子 token 化，并将输入转移到 GPU
    inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # 转移到 GPU

    # 获取BERT的输出
    with torch.no_grad():  # 禁用梯度计算，节省内存
        outputs = model(**inputs)

    # BERT的最后隐藏层的输出 [batch_size, seq_len, hidden_size]
    last_hidden_states = outputs.last_hidden_state

    # 取 [CLS] token 的 embedding（句子嵌入）
    cls_embedding = last_hidden_states[:, 0, :]

    # 也可以取所有 token 的平均值作为句子嵌入
    mean_embedding = torch.mean(last_hidden_states, dim=1)

    print(f"CLS token embedding size: {cls_embedding.shape}")
    print(f"Mean token embedding size: {mean_embedding.shape}")

    return cls_embedding,mean_embedding

In [ ]:
# SCIB_cls_embedding,SCIB_mean_embedding=SCIBERT(text_list)
SCIB_cls_embedding1,SCIB_mean_embedding1=SCIBERT(text_list[:1000])
SCIB_cls_embedding2,SCIB_mean_embedding2=SCIBERT(text_list[1000:2000])
SCIB_cls_embedding3,SCIB_mean_embedding3=SCIBERT(text_list[2000:3000])
SCIB_cls_embedding4,SCIB_mean_embedding4=SCIBERT(text_list[3000:])

SCIB_cls_embedding=torch.cat((SCIB_cls_embedding1,SCIB_cls_embedding2,SCIB_cls_embedding3,SCIB_cls_embedding4))
SCIB_mean_embedding=torch.cat((SCIB_mean_embedding1,SCIB_mean_embedding2,SCIB_mean_embedding3,SCIB_mean_embedding4))



In [ ]:
SCIB_cls_embedding.shape,SCIB_mean_embedding.shape

In [ ]:
# SCIB_extracted_text_cls_embedding,SCIB_extracted_text_mean_embedding=SCIBERT(et_list)
SCIB_extracted_text_cls_embedding1,SCIB_extracted_text_mean_embedding1=SCIBERT(et_list[:1000])
SCIB_extracted_text_cls_embedding2,SCIB_extracted_text_mean_embedding2=SCIBERT(et_list[1000:2000])
SCIB_extracted_text_cls_embedding3,SCIB_extracted_text_mean_embedding3=SCIBERT(et_list[2000:3000])
SCIB_extracted_text_cls_embedding4,SCIB_extracted_text_mean_embedding4=SCIBERT(et_list[3000:])

SCIB_extracted_text_cls_embedding=torch.cat((SCIB_extracted_text_cls_embedding1,SCIB_extracted_text_cls_embedding2,SCIB_extracted_text_cls_embedding3,SCIB_extracted_text_cls_embedding4))
SCIB_extracted_text_mean_embedding=torch.cat((SCIB_extracted_text_mean_embedding1,SCIB_extracted_text_mean_embedding2,SCIB_extracted_text_mean_embedding3,SCIB_extracted_text_mean_embedding4))

In [ ]:
de_biased_cka(SCIB_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(SCIB_cls_embedding[compute_list], gcn_out_embeddings),de_biased_cka(SCIB_cls_embedding[compute_list],gin_embeddings),de_biased_cka(SCIB_cls_embedding[compute_list],gin_out_embeddings)
# (0.22608945, 0.21755558, 0.19078067, 0.21649921)
# (0.22608945, 0.21755558, 0.22363263, 0.22730517)
# (0.22608945, 0.21755558, 0.22283247, 0.21325558)
# (0.22608945, 0.21755558, 0.1987281, 0.19862846)

In [ ]:
de_biased_cka(SCIB_mean_embedding[compute_list],gcn_embeddings),de_biased_cka(SCIB_mean_embedding[compute_list],gcn_out_embeddings),de_biased_cka(SCIB_mean_embedding[compute_list],gin_embeddings),de_biased_cka(SCIB_mean_embedding[compute_list],gin_out_embeddings)
# (0.24103628, 0.24200632, 0.18887858, 0.2180085)
# (0.24103628, 0.24200632, 0.22267848, 0.22766191)
# (0.24103628, 0.24200632, 0.22232172, 0.21107413)
# (0.24103628, 0.24200632, 0.19761267, 0.19851111)

In [ ]:
de_biased_cka(SCIB_extracted_text_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(SCIB_extracted_text_cls_embedding[compute_list],gcn_out_embeddings),de_biased_cka(SCIB_extracted_text_cls_embedding[compute_list],gin_embeddings),de_biased_cka(SCIB_extracted_text_cls_embedding[compute_list],gin_out_embeddings)
# (0.2045951, 0.20098218, 0.18605205, 0.18484423)
# (0.2045951, 0.20098218, 0.17631763, 0.18533646)
# (0.2045951, 0.20098218, 0.17070535, 0.17068122)

In [ ]:
de_biased_cka(SCIB_extracted_text_mean_embedding[compute_list], gcn_embeddings),de_biased_cka(SCIB_extracted_text_mean_embedding[compute_list],gcn_out_embeddings),de_biased_cka(SCIB_extracted_text_mean_embedding[compute_list],gin_embeddings),de_biased_cka(SCIB_extracted_text_mean_embedding[compute_list],gin_out_embeddings)
# (0.2992024, 0.29717383, 0.2547181, 0.25347224)
# (0.2992024, 0.29717383, 0.24284594, 0.2569317)
# (0.2992024, 0.29717383, 0.2356572, 0.23578407)

In [ ]:
print(CKA(SCIB_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(SCIB_cls_embedding[compute_list], gcn_out_embeddings.to(device)),CKA(SCIB_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(SCIB_cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(SCIB_mean_embedding[compute_list],gcn_embeddings.to(device)),CKA(SCIB_mean_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(SCIB_mean_embedding[compute_list],gin_embeddings.to(device)),CKA(SCIB_mean_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(SCIB_extracted_text_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(SCIB_extracted_text_cls_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(SCIB_extracted_text_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(SCIB_extracted_text_cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(SCIB_extracted_text_mean_embedding[compute_list], gcn_embeddings.to(device)),CKA(SCIB_extracted_text_mean_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(SCIB_extracted_text_mean_embedding[compute_list],gin_embeddings.to(device)),CKA(SCIB_extracted_text_mean_embedding[compute_list],gin_out_embeddings.to(device)))

# tensor(0.2303, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.2236, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.2118, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.2206, device='cuda:0', grad_fn=<DivBackward0>)
# tensor(0.2463, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.2491, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.2126, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.2251, device='cuda:0', grad_fn=<DivBackward0>)
# tensor(0.2112, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.2102, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.1844, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.1938, device='cuda:0', grad_fn=<DivBackward0>)
# tensor(0.3025, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.3028, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.2488, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.2630, device='cuda:0', grad_fn=<DivBackward0>)

In [ ]:
len(text_list),len(et_list)

In [ ]:
del SCIB_cls_embedding1,
del SCIB_mean_embedding1,
del SCIB_cls_embedding2,
del SCIB_mean_embedding2,
del SCIB_cls_embedding3,
del SCIB_mean_embedding3,
del SCIB_cls_embedding4,
del SCIB_mean_embedding4,
del SCIB_cls_embedding,
del SCIB_mean_embedding,
del SCIB_extracted_text_cls_embedding1,
del SCIB_extracted_text_mean_embedding1,
del SCIB_extracted_text_cls_embedding2,
del SCIB_extracted_text_mean_embedding2,
del SCIB_extracted_text_cls_embedding3,
del SCIB_extracted_text_mean_embedding3,
del SCIB_extracted_text_cls_embedding4,
del SCIB_extracted_text_mean_embedding4,
del SCIB_extracted_text_cls_embedding,
del SCIB_extracted_text_mean_embedding,

import gc
gc.collect()
torch.cuda.empty_cache()

## Sentence Bert

In [ ]:
torch.cuda.empty_cache()
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("distiluse-base-multilingual-cased-v2").to(device)
# 2. Calculate embeddings by calling model.encode()
SB_embeddings = model.encode(text_list)
print('embeddings',SB_embeddings.shape)

SB_extracted_embeddings = model.encode(et_list)
print('extracted_embeddings',SB_extracted_embeddings.shape)

In [ ]:
de_biased_cka(torch.from_numpy(SB_embeddings)[compute_list],gcn_embeddings),de_biased_cka(torch.from_numpy(SB_extracted_embeddings)[compute_list],gcn_embeddings),de_biased_cka(torch.from_numpy(SB_embeddings)[compute_list],gcn_out_embeddings),de_biased_cka(torch.from_numpy(SB_extracted_embeddings)[compute_list],gcn_out_embeddings)
# (0.2921439, 0.26299226, 0.28667632, 0.25718084)
# (0.2921439, 0.26299226, 0.28667632, 0.25718084)

In [ ]:
de_biased_cka(torch.from_numpy(SB_embeddings)[compute_list],gin_embeddings),de_biased_cka(torch.from_numpy(SB_extracted_embeddings)[compute_list],gin_embeddings),de_biased_cka(torch.from_numpy(SB_embeddings)[compute_list],gin_out_embeddings),de_biased_cka(torch.from_numpy(SB_extracted_embeddings)[compute_list],gin_out_embeddings)
# (0.25000453, 0.23379803, 0.2507004, 0.23436297)
# (0.26438615, 0.2464931, 0.28694633, 0.26617283)
# (0.2528803, 0.23645988, 0.25703993, 0.23996557)
# (0.26420167, 0.24588796, 0.2621179, 0.24466914)

In [ ]:
print(CKA(torch.from_numpy(SB_embeddings)[compute_list],gcn_embeddings),CKA(torch.from_numpy(SB_extracted_embeddings)[compute_list],gcn_embeddings),CKA(torch.from_numpy(SB_embeddings)[compute_list],gcn_out_embeddings),CKA(torch.from_numpy(SB_extracted_embeddings)[compute_list],gcn_out_embeddings))
print(CKA(torch.from_numpy(SB_embeddings)[compute_list],gin_embeddings),CKA(torch.from_numpy(SB_extracted_embeddings)[compute_list],gin_embeddings),CKA(torch.from_numpy(SB_embeddings)[compute_list],gin_out_embeddings),CKA(torch.from_numpy(SB_extracted_embeddings)[compute_list],gin_out_embeddings))

In [ ]:
print(CKA(torch.from_numpy(SB_embeddings)[compute_list],gcn_embeddings),CKA(torch.from_numpy(SB_extracted_embeddings)[compute_list],gcn_embeddings),CKA(torch.from_numpy(SB_embeddings)[compute_list],gcn_out_embeddings),CKA(torch.from_numpy(SB_extracted_embeddings)[compute_list],gcn_out_embeddings))
print(CKA(torch.from_numpy(SB_embeddings)[compute_list],gin_embeddings),CKA(torch.from_numpy(SB_extracted_embeddings)[compute_list],gin_embeddings),CKA(torch.from_numpy(SB_embeddings)[compute_list],gin_out_embeddings),CKA(torch.from_numpy(SB_extracted_embeddings)[compute_list],gin_out_embeddings))

### match

In [ ]:
K=10
# GCN
SB_gcn_texts_match_indices, SB_texts_match_scores, SB_texts_match_correct_total = retrieve(gcn_embeddings.cpu(), torch.from_numpy(SB_embeddings).cpu().detach()[compute_list], K=10)
SB_gcn_extracted_match_indices, SB_extracted_match_scores, SB_extracted_match_correct_total = retrieve(gcn_embeddings.cpu(), torch.from_numpy(SB_extracted_embeddings).cpu().detach()[compute_list], K=10)

print(SB_texts_match_correct_total,SB_extracted_match_correct_total)

In [ ]:
K=10
# GIN
SB_gin_texts_match_indices, SB_texts_match_scores, SB_texts_match_correct_total = retrieve(gin_embeddings.cpu(), torch.from_numpy(SB_embeddings).cpu().detach()[compute_list], K=10)
SB_gin_extracted_match_indices, SB_extracted_match_scores, SB_extracted_match_correct_total = retrieve(gin_embeddings.cpu(), torch.from_numpy(SB_extracted_embeddings).cpu().detach()[compute_list], K=10)

print(SB_texts_match_correct_total,SB_extracted_match_correct_total)

In [ ]:
K=1
# GCN
SB_gcn_texts_k1_match_indices, SB_texts_k1_match_scores, SB_texts_k1_match_correct_total = retrieve(gcn_embeddings.cpu(), torch.from_numpy(SB_embeddings).cpu().detach()[compute_list], K=K)
SB_gcn_extracte_k1_match_indices, SB_extracted_k1_match_scores, SB_extracted_k1_match_correct_total = retrieve(gcn_embeddings.cpu(), torch.from_numpy(SB_extracted_embeddings).cpu().detach()[compute_list], K=K)

print(SB_texts_k1_match_correct_total,SB_extracted_k1_match_correct_total)

In [ ]:
K=1
# GIN
SB_gin_texts_k1_match_indices, SB_texts_match_scores, SB_texts_match_correct_total = retrieve(gin_embeddings.cpu(), torch.from_numpy(SB_embeddings).cpu().detach()[compute_list], K=K)
SB_gin_extracted_k1_match_indices, SB_extracted_match_scores, SB_extracted_match_correct_total = retrieve(gin_embeddings.cpu(), torch.from_numpy(SB_extracted_embeddings).cpu().detach()[compute_list], K=K)

print(SB_texts_match_correct_total,SB_extracted_match_correct_total)

### text NN

In [ ]:
SB_texts_mean_NN=NN(embedding=torch.from_numpy(SB_embeddings)[compute_list])
SB_extracted_text_mean_NN=NN(embedding=torch.from_numpy(SB_extracted_embeddings)[compute_list])

In [ ]:
SB_emb_distance,SB_gcn_emb_index=SB_texts_mean_NN.kneighbors(SB_embeddings[compute_list])
SB_et_distance,SB_gcn_et_index=SB_extracted_text_mean_NN.kneighbors(SB_extracted_embeddings[compute_list])

SB_emb_distance,SB_gin_emb_index=SB_texts_mean_NN.kneighbors(SB_embeddings[compute_list])
SB_et_distance,SB_gin_et_index=SB_extracted_text_mean_NN.kneighbors(SB_extracted_embeddings[compute_list])

In [ ]:
overleap(SB_gcn_texts_match_indices.cpu().numpy(), SB_gcn_emb_index)

overleap(SB_gcn_extracted_match_indices.cpu().numpy(), SB_gcn_et_index)

overleap(SB_gin_texts_match_indices.cpu().numpy(), SB_gin_emb_index)

overleap(SB_gin_extracted_match_indices.cpu().numpy(), SB_gin_et_index)

### SMILES NN

In [ ]:
SB_GCN_NN=NN(embedding=gcn_embeddings[compute_list])
SB_GIN_NN=NN(embedding=gin_embeddings[compute_list])

In [ ]:
SB_emb_distance,SB_gcn_emb_index=SB_GCN_NN.kneighbors(gcn_embeddings.detach().numpy()[compute_list])
SB_et_distance,SB_gcn_et_index=SB_GCN_NN.kneighbors(gcn_embeddings.detach().numpy()[compute_list])

SB_emb_distance,SB_gin_emb_index=SB_GIN_NN.kneighbors(gin_embeddings.detach().numpy()[compute_list])
SB_et_distance,SB_gin_et_index=SB_GIN_NN.kneighbors(gin_embeddings.detach().numpy()[compute_list])

In [ ]:
overleap(SB_gcn_texts_match_indices.cpu().numpy(), SB_gcn_emb_index)

overleap(SB_gcn_extracted_match_indices.cpu().numpy(), SB_gcn_et_index)

overleap(SB_gin_texts_match_indices.cpu().numpy(), SB_gin_emb_index)

overleap(SB_gin_extracted_match_indices.cpu().numpy(), SB_gin_et_index)

### text K-means

In [ ]:
SB_cls_KM=KM(SB_embeddings[compute_list])

SB_extracted_text_KM=KM(SB_extracted_embeddings[compute_list])

In [ ]:
SB_cls=SB_cls_KM.predict(SB_embeddings[compute_list])
SB_extracted_text_cls=SB_extracted_text_KM.predict(SB_extracted_embeddings[compute_list])

In [ ]:
SB_cls_index=SB_cls_KM.predict(SB_embeddings[SB_gcn_texts_k1_match_indices].reshape(SB_embeddings.shape)[compute_list])
SB_et_index=SB_extracted_text_KM.predict(SB_extracted_embeddings[SB_gcn_extracte_k1_match_indices].reshape(SB_extracted_embeddings.shape)[compute_list])

In [ ]:
SB_x_cls=SB_cls_index==SB_cls
SB_et=SB_et_index==SB_extracted_text_cls
np.sum(SB_x_cls),np.sum(SB_et)

In [ ]:
SB_cls_index=SB_cls_KM.predict(SB_embeddings[SB_gin_texts_k1_match_indices].reshape(SB_embeddings.shape)[compute_list])
SB_et_index=SB_extracted_text_KM.predict(SB_extracted_embeddings[SB_gin_extracted_k1_match_indices].reshape(SB_extracted_embeddings.shape)[compute_list])

In [ ]:
SB_x_cls=SB_cls_index==SB_cls
SB_et=SB_et_index==SB_extracted_text_cls
np.sum(SB_x_cls),np.sum(SB_et)

### SMIS K-means

In [ ]:
SBSMIS_cls_KM=KM(gcn_embeddings.detach().numpy())

SBSMIS_extracted_text_KM=KM(gcn_embeddings.detach().numpy())

In [ ]:
SBSMIS_cls=SBSMIS_cls_KM.predict(gcn_embeddings.detach().numpy())
SBSMIS_extracted_text_cls=SBSMIS_extracted_text_KM.predict(gcn_embeddings.detach().numpy())

In [ ]:
SBSMIS_cls_index=SBSMIS_cls_KM.predict(gcn_embeddings[SB_gcn_texts_k1_match_indices].reshape(gcn_embeddings.shape).detach().numpy() )
SBSMIS_et_index=SBSMIS_extracted_text_KM.predict(gcn_embeddings[SB_gcn_extracte_k1_match_indices].reshape(gcn_embeddings.shape).detach().numpy() )

In [ ]:
SBSMIS_x_cls=SBSMIS_cls_index==SBSMIS_cls
SBSMIS_et=SBSMIS_et_index==SBSMIS_extracted_text_cls
np.sum(SBSMIS_x_cls),np.sum(SBSMIS_et)

### GIN

In [ ]:
SBSMIS_cls_KM=KM(gin_embeddings.detach().numpy())
SBSMIS_extracted_text_KM=KM(gin_embeddings.detach().numpy())

In [ ]:
SBSMIS_cls=SBSMIS_cls_KM.predict(gin_embeddings.detach().numpy())
SBSMIS_extracted_text_cls=SBSMIS_extracted_text_KM.predict(gin_embeddings.detach().numpy())

In [ ]:
SBSMIS_cls_index=SBSMIS_cls_KM.predict(gin_embeddings[SB_gin_texts_k1_match_indices].reshape(gcn_embeddings.shape).detach().numpy())
SBSMIS_et_index=SBSMIS_extracted_text_KM.predict(gin_embeddings[SB_gin_extracted_k1_match_indices].reshape(gcn_embeddings.shape).detach().numpy() )

In [ ]:
SBSMIS_x_cls=SBSMIS_cls_index==SBSMIS_cls
SBSMIS_et=SBSMIS_et_index==SBSMIS_extracted_text_cls
np.sum(SBSMIS_x_cls),np.sum(SBSMIS_et)

In [ ]:
del SB_embeddings,
del SB_extracted_embeddings,

import gc
gc.collect()
torch.cuda.empty_cache()

## Pure BERT

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
def pureBert(texts=texts):
    # 检查是否有可用的 GPU
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    # device = torch.device('cuda:1')
    # 加载预训练的BERT模型和tokenizer
    model_name = 'bert-base-uncased'  # 你可以使用其他的BERT模型，例如 'bert-large-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    # 将模型转移到 GPU
    model = model.to(device)

    # 将句子 token 化，并将输入转移到 GPU
    inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # 转移到 GPU

    # 获取BERT的输出
    with torch.no_grad():  # 禁用梯度计算，节省内存
        outputs = model(**inputs)

    # BERT的最后隐藏层的输出 [batch_size, seq_len, hidden_size]
    last_hidden_states = outputs.last_hidden_state

    # 取 [CLS] token 的 embedding（句子嵌入）
    cls_embedding = last_hidden_states[:, 0, :]

    # 也可以取所有 token 的平均值作为句子嵌入
    mean_embedding = torch.mean(last_hidden_states, dim=1)

    print(f"CLS token embedding size: {cls_embedding.shape}")
    print(f"Mean token embedding size: {mean_embedding.shape}")

    return cls_embedding,mean_embedding


In [ ]:
# PB_cls_embedding,PB_mean_embedding=pureBert(text_list)
PB_cls_embedding1,PB_mean_embedding1=pureBert(text_list[:1000])
PB_cls_embedding2,PB_mean_embedding2=pureBert(text_list[1000:2000])
PB_cls_embedding3,PB_mean_embedding3=pureBert(text_list[2000:3000])
PB_cls_embedding4,PB_mean_embedding4=pureBert(text_list[3000:])

PB_cls_embedding=torch.cat((PB_cls_embedding1,PB_cls_embedding2,PB_cls_embedding3,PB_cls_embedding4))
PB_mean_embedding=torch.cat((PB_mean_embedding1,PB_mean_embedding2,PB_mean_embedding3,PB_mean_embedding4))

In [ ]:
PB_cls_embedding.shape,PB_mean_embedding.shape

In [ ]:
# PB_extracted_text_cls_embedding,PB_extracted_text_mean_embedding=pureBert(et_list)
PB_extracted_text_cls_embedding1,PB_extracted_text_mean_embedding1=pureBert(et_list[:1000])
PB_extracted_text_cls_embedding2,PB_extracted_text_mean_embedding2=pureBert(et_list[1000:2000])
PB_extracted_text_cls_embedding3,PB_extracted_text_mean_embedding3=pureBert(et_list[2000:3000])
PB_extracted_text_cls_embedding4,PB_extracted_text_mean_embedding4=pureBert(et_list[3000:])

PB_extracted_text_cls_embedding=torch.cat((PB_extracted_text_cls_embedding1,PB_extracted_text_cls_embedding2,PB_extracted_text_cls_embedding3,PB_extracted_text_cls_embedding4))
PB_extracted_text_mean_embedding=torch.cat((PB_extracted_text_mean_embedding1,PB_extracted_text_mean_embedding2,PB_extracted_text_mean_embedding3,PB_extracted_text_mean_embedding4))

In [ ]:
de_biased_cka(PB_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(PB_cls_embedding[compute_list], gcn_out_embeddings),de_biased_cka(PB_cls_embedding[compute_list],gin_embeddings),de_biased_cka(PB_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(PB_mean_embedding[compute_list],gcn_embeddings),de_biased_cka(PB_mean_embedding[compute_list],gcn_out_embeddings),de_biased_cka(PB_mean_embedding[compute_list],gin_embeddings),de_biased_cka(PB_mean_embedding[compute_list],gin_out_embeddings)
# (0.09211786, 0.089327276, 0.09924333, 0.098099664)

In [ ]:
de_biased_cka(PB_extracted_text_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(PB_extracted_text_cls_embedding[compute_list],gcn_out_embeddings),de_biased_cka(PB_extracted_text_cls_embedding[compute_list],gin_embeddings),de_biased_cka(PB_extracted_text_cls_embedding[compute_list],gin_out_embeddings)
# (0.14848486, 0.14544165, 0.14571217, 0.14427526)

In [ ]:
de_biased_cka(PB_extracted_text_mean_embedding[compute_list], gcn_embeddings),de_biased_cka(PB_extracted_text_mean_embedding[compute_list],gcn_out_embeddings),de_biased_cka(PB_extracted_text_mean_embedding[compute_list],gin_embeddings),de_biased_cka(PB_extracted_text_mean_embedding[compute_list],gin_out_embeddings)
# (0.1153796, 0.11176958, 0.11518018, 0.11434773)

In [ ]:
print(CKA(PB_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(PB_cls_embedding[compute_list], gcn_out_embeddings.to(device)),CKA(PB_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(PB_cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(PB_mean_embedding[compute_list],gcn_embeddings.to(device)),CKA(PB_mean_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(PB_mean_embedding[compute_list],gin_embeddings.to(device)),CKA(PB_mean_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(PB_extracted_text_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(PB_extracted_text_cls_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(PB_extracted_text_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(PB_extracted_text_cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(PB_extracted_text_mean_embedding[compute_list], gcn_embeddings.to(device)),CKA(PB_extracted_text_mean_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(PB_extracted_text_mean_embedding[compute_list],gin_embeddings.to(device)),CKA(PB_extracted_text_mean_embedding[compute_list],gin_out_embeddings.to(device)))
# tensor(0.1656, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.1650, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.1629, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.1613, device='cuda:0', grad_fn=<DivBackward0>)
# tensor(0.0987, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.0975, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.1058, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.1047, device='cuda:0', grad_fn=<DivBackward0>)
# tensor(0.1557, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.1550, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.1533, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.1520, device='cuda:0', grad_fn=<DivBackward0>)
# tensor(0.1221, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.1203, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.1222, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.1214, device='cuda:0', grad_fn=<DivBackward0>)

In [ ]:
len(text_list),len(et_list)

In [ ]:
del PB_cls_embedding1,
del PB_mean_embedding1,
del PB_cls_embedding2,
del PB_mean_embedding2,
del PB_cls_embedding3,
del PB_mean_embedding3,
del PB_cls_embedding4,
del PB_mean_embedding4,
del PB_cls_embedding,
del PB_mean_embedding,
del PB_extracted_text_cls_embedding1,
del PB_extracted_text_mean_embedding1,
del PB_extracted_text_cls_embedding2,
del PB_extracted_text_mean_embedding2,
del PB_extracted_text_cls_embedding3,
del PB_extracted_text_mean_embedding3,
del PB_extracted_text_cls_embedding4,
del PB_extracted_text_mean_embedding4,
del PB_extracted_text_cls_embedding,
del PB_extracted_text_mean_embedding,

import gc
gc.collect()
torch.cuda.empty_cache()

## GPT

In [ ]:
from transformers import AutoTokenizer, OpenAIGPTModel
import torch
def get_GPTembedding(texts=texts):
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("openai-community/openai-gpt")
    model = OpenAIGPTModel.from_pretrained("openai-community/openai-gpt")

    # Set the padding token
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))

    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenize input texts
    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True)

    # Move inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Run model to get hidden states
    with torch.no_grad():  # Disable gradient calculation for inference
        outputs = model(**inputs)

    # Access last hidden states
    last_hidden_states = outputs.last_hidden_state
    print(last_hidden_states.shape)

    # 获取最后的隐藏状态 [batch_size, seq_len, hidden_size]
    last_hidden_states = outputs.last_hidden_state

    # 获取第一个 token (<s> token) 的 embedding（相当于 CLS token 的句子嵌入）
    cls_embedding = last_hidden_states[:, 0, :]

    # 也可以取所有 token 的平均值作为句子嵌入
    mean_embedding = torch.mean(last_hidden_states, dim=1)

    print(f"CLS token embedding size: {cls_embedding.shape}")
    print(f"Mean token embedding size: {mean_embedding.shape}")
    return cls_embedding,mean_embedding


In [ ]:
# GPT_cls_embedding,GPT_mean_embedding=get_GPTembedding(text_list)
GPT_cls_embedding1,GPT_mean_embedding1=get_GPTembedding(text_list[:1000])
GPT_cls_embedding2,GPT_mean_embedding2=get_GPTembedding(text_list[1000:2000])
GPT_cls_embedding3,GPT_mean_embedding3=get_GPTembedding(text_list[2000:3000])
GPT_cls_embedding4,GPT_mean_embedding4=get_GPTembedding(text_list[3000:])

GPT_cls_embedding=torch.cat((GPT_cls_embedding1,GPT_cls_embedding2,GPT_cls_embedding3,GPT_cls_embedding4))
GPT_mean_embedding=torch.cat((GPT_mean_embedding1,GPT_mean_embedding2,GPT_mean_embedding3,GPT_mean_embedding4))

In [ ]:
GPT_cls_embedding.shape,GPT_mean_embedding.shape

In [ ]:
# GPT_extracted_text_cls_embedding,GPT_extracted_text_mean_embedding=get_GPTembedding(et_list)
GPT_extracted_text_cls_embedding1,GPT_extracted_text_mean_embedding1=get_GPTembedding(et_list[:1000])
GPT_extracted_text_cls_embedding2,GPT_extracted_text_mean_embedding2=get_GPTembedding(et_list[1000:2000])
GPT_extracted_text_cls_embedding3,GPT_extracted_text_mean_embedding3=get_GPTembedding(et_list[2000:3000])
GPT_extracted_text_cls_embedding4,GPT_extracted_text_mean_embedding4=get_GPTembedding(et_list[3000:])

GPT_extracted_text_cls_embedding=torch.cat((GPT_extracted_text_cls_embedding1,GPT_extracted_text_cls_embedding2,GPT_extracted_text_cls_embedding3,GPT_extracted_text_cls_embedding4))
GPT_extracted_text_mean_embedding=torch.cat((GPT_extracted_text_mean_embedding1,GPT_extracted_text_mean_embedding2,GPT_extracted_text_mean_embedding3,GPT_extracted_text_mean_embedding4))

### CKA

In [ ]:
de_biased_cka(GPT_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(GPT_cls_embedding[compute_list], gcn_out_embeddings),de_biased_cka(GPT_cls_embedding[compute_list],gin_embeddings),de_biased_cka(GPT_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(GPT_mean_embedding[compute_list],gcn_embeddings),de_biased_cka(GPT_mean_embedding[compute_list],gcn_out_embeddings),de_biased_cka(GPT_mean_embedding[compute_list],gin_embeddings),de_biased_cka(GPT_mean_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(GPT_extracted_text_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(GPT_extracted_text_cls_embedding[compute_list],gcn_out_embeddings),de_biased_cka(GPT_extracted_text_cls_embedding[compute_list],gin_embeddings),de_biased_cka(GPT_extracted_text_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(GPT_extracted_text_mean_embedding[compute_list], gcn_embeddings),de_biased_cka(GPT_extracted_text_mean_embedding[compute_list],gcn_out_embeddings),de_biased_cka(GPT_extracted_text_mean_embedding[compute_list],gin_embeddings),de_biased_cka(GPT_extracted_text_mean_embedding[compute_list],gin_out_embeddings)

In [ ]:
print(CKA(GPT_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(GPT_cls_embedding[compute_list], gcn_out_embeddings.to(device)),CKA(GPT_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(GPT_cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(GPT_mean_embedding[compute_list],gcn_embeddings.to(device)),CKA(GPT_mean_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(GPT_mean_embedding[compute_list],gin_embeddings.to(device)),CKA(GPT_mean_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(GPT_extracted_text_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(GPT_extracted_text_cls_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(GPT_extracted_text_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(GPT_extracted_text_cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(GPT_extracted_text_mean_embedding[compute_list], gcn_embeddings.to(device)),CKA(GPT_extracted_text_mean_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(GPT_extracted_text_mean_embedding[compute_list],gin_embeddings.to(device)),CKA(GPT_extracted_text_mean_embedding[compute_list],gin_out_embeddings.to(device)))

In [ ]:
del GPT_cls_embedding1,
del GPT_mean_embedding1,
del GPT_cls_embedding2,
del GPT_mean_embedding2,
del GPT_cls_embedding3,
del GPT_mean_embedding3,
del GPT_cls_embedding4,
del GPT_mean_embedding4,
del GPT_cls_embedding,
del GPT_mean_embedding,
del GPT_extracted_text_cls_embedding1,
del GPT_extracted_text_mean_embedding1,
del GPT_extracted_text_cls_embedding2,
del GPT_extracted_text_mean_embedding2,
del GPT_extracted_text_cls_embedding3,
del GPT_extracted_text_mean_embedding3,
del GPT_extracted_text_cls_embedding4,
del GPT_extracted_text_mean_embedding4,
del GPT_extracted_text_cls_embedding,
del GPT_extracted_text_mean_embedding,

import gc
gc.collect()
torch.cuda.empty_cache()

## last word GPT

In [ ]:
import torch
from transformers import AutoTokenizer, OpenAIGPTModel

def LastGPTembedding(texts):

    # Check if GPU is available and set the device
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Load pre-trained model and tokenizer and move the model to the GPU
    tokenizer = AutoTokenizer.from_pretrained("openai-community/openai-gpt")
    model = OpenAIGPTModel.from_pretrained("openai-community/openai-gpt").to(device)
    
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))
    # Input sentences
    # texts = ['Your sentence here1', 'Your sentence here2']

    # Tokenize input sentence and move input tensors to the GPU

    template = 'This_sentence_:_"The_molecule_is_a_nitrile_that_is_acetonitrile_where_one_of_the_methyl_hydrogens_is_substituted_by_a_2-methylphenyl_group."_means_in_one_word:"Acetonitrile".This_sentence_:_"*sent_0*"_means_in_one_word:"'
    inputs = tokenizer([template.replace('*sent_0*', i).replace('_', ' ') for i in texts], padding=True, truncation=True,  max_length=512,return_tensors="pt")['input_ids'].to(device)
    with torch.no_grad():
        embeddings = model(inputs, output_hidden_states=True, return_dict=True).hidden_states[-1][:, -1, :]
    print(embeddings.shape)
    return embeddings



In [ ]:
# LastGPT_cls_embedding=LastGPTembedding(text_list)

LastGPT_cls_embedding1=LastGPTembedding(text_list[:1000])
LastGPT_cls_embedding2=LastGPTembedding(text_list[1000:2000])
LastGPT_cls_embedding3=LastGPTembedding(text_list[2000:3000])
LastGPT_cls_embedding4=LastGPTembedding(text_list[3000:])

LastGPT_cls_embedding=torch.cat((LastGPT_cls_embedding1,LastGPT_cls_embedding2,LastGPT_cls_embedding3,LastGPT_cls_embedding4))

In [ ]:
# LastGPT_extracted_text_cls_embedding=LastGPTembedding(et_list)

LastGPT_extracted_text_cls_embedding1=LastGPTembedding(et_list[:1000])
LastGPT_extracted_text_cls_embedding2=LastGPTembedding(et_list[1000:2000])
LastGPT_extracted_text_cls_embedding3=LastGPTembedding(et_list[2000:3000])
LastGPT_extracted_text_cls_embedding4=LastGPTembedding(et_list[3000:])

LastGPT_extracted_text_cls_embedding=torch.cat((LastGPT_extracted_text_cls_embedding1,LastGPT_extracted_text_cls_embedding2,LastGPT_extracted_text_cls_embedding3,LastGPT_extracted_text_cls_embedding4))

### CKA

In [ ]:
de_biased_cka(LastGPT_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(LastGPT_cls_embedding[compute_list], gcn_out_embeddings),de_biased_cka(LastGPT_cls_embedding[compute_list],gin_embeddings),de_biased_cka(LastGPT_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(LastGPT_extracted_text_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(LastGPT_extracted_text_cls_embedding[compute_list],gcn_out_embeddings),de_biased_cka(LastGPT_extracted_text_cls_embedding[compute_list],gin_embeddings),de_biased_cka(LastGPT_extracted_text_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
print(CKA(LastGPT_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(LastGPT_cls_embedding[compute_list], gcn_out_embeddings.to(device)),CKA(LastGPT_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(LastGPT_cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(LastGPT_extracted_text_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(LastGPT_extracted_text_cls_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(LastGPT_extracted_text_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(LastGPT_extracted_text_cls_embedding[compute_list],gin_out_embeddings.to(device)))

In [ ]:
del LastGPT_cls_embedding1,
del LastGPT_cls_embedding2,
del LastGPT_cls_embedding3,
del LastGPT_cls_embedding4,
del LastGPT_cls_embedding,
del LastGPT_extracted_text_cls_embedding1,
del LastGPT_extracted_text_cls_embedding2,
del LastGPT_extracted_text_cls_embedding3,
del LastGPT_extracted_text_cls_embedding4,
del LastGPT_extracted_text_cls_embedding,


import gc
gc.collect()
torch.cuda.empty_cache()

## Summarise last word GPT

In [ ]:
import torch
from transformers import AutoTokenizer, OpenAIGPTModel

def LastSGPTembedding(texts):

    # Check if GPU is available and set the device
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Load pre-trained model and tokenizer and move the model to the GPU
    tokenizer = AutoTokenizer.from_pretrained("openai-community/openai-gpt")
    model = OpenAIGPTModel.from_pretrained("openai-community/openai-gpt").to(device)
    
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))
    # Input sentences
    # texts = ['Your sentence here1', 'Your sentence here2']

    # Tokenize input sentence and move input tensors to the GPU

    template = 'This_sentence_:_"The_molecule_is_a_nitrile_that_is_acetonitrile_where_one_of_the_methyl_hydrogens_is_substituted_by_a_2-methylphenyl_group."_summarise_in_one_word:"Acetonitrile".This_sentence_:_"*sent_0*"_summarise_in_one_word:"'
    inputs = tokenizer([template.replace('*sent_0*', i).replace('_', ' ') for i in texts], padding=True, truncation=True,  max_length=512,return_tensors="pt")['input_ids'].to(device)
    with torch.no_grad():
        embeddings = model(inputs, output_hidden_states=True, return_dict=True).hidden_states[-1][:, -1, :]
    print(embeddings.shape)
    return embeddings



In [ ]:
# LastSGPT_cls_embedding=LastSGPTembedding(text_list)
LastSGPT_cls_embedding1=LastSGPTembedding(text_list[:1000])
LastSGPT_cls_embedding2=LastSGPTembedding(text_list[1000:2000])
LastSGPT_cls_embedding3=LastSGPTembedding(text_list[2000:3000])
LastSGPT_cls_embedding4=LastSGPTembedding(text_list[3000:])

LastSGPT_cls_embedding=torch.cat((LastSGPT_cls_embedding1,LastSGPT_cls_embedding2,LastSGPT_cls_embedding3,LastSGPT_cls_embedding4))


In [ ]:
# LastSGPT_extracted_text_cls_embedding=LastSGPTembedding(et_list)
LastSGPT_extracted_text_cls_embedding1=LastGPTembedding(et_list[:1000])
LastSGPT_extracted_text_cls_embedding2=LastGPTembedding(et_list[1000:2000])
LastSGPT_extracted_text_cls_embedding3=LastGPTembedding(et_list[2000:3000])
LastSGPT_extracted_text_cls_embedding4=LastGPTembedding(et_list[3000:])

LastSGPT_extracted_text_cls_embedding=torch.cat((LastSGPT_extracted_text_cls_embedding1,LastSGPT_extracted_text_cls_embedding2,LastSGPT_extracted_text_cls_embedding3,LastSGPT_extracted_text_cls_embedding4))

### CKA

In [ ]:
de_biased_cka(LastSGPT_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(LastSGPT_cls_embedding[compute_list], gcn_out_embeddings),de_biased_cka(LastSGPT_cls_embedding[compute_list],gin_embeddings),de_biased_cka(LastSGPT_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(LastSGPT_extracted_text_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(LastSGPT_extracted_text_cls_embedding[compute_list],gcn_out_embeddings),de_biased_cka(LastSGPT_extracted_text_cls_embedding[compute_list],gin_embeddings),de_biased_cka(LastSGPT_extracted_text_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
print(CKA(LastSGPT_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(LastSGPT_cls_embedding[compute_list], gcn_out_embeddings.to(device)),CKA(LastSGPT_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(LastSGPT_cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(LastSGPT_extracted_text_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(LastSGPT_extracted_text_cls_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(LastSGPT_extracted_text_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(LastSGPT_extracted_text_cls_embedding[compute_list],gin_out_embeddings.to(device)))

In [ ]:
del LastSGPT_cls_embedding1,
del LastSGPT_cls_embedding2,
del LastSGPT_cls_embedding3,
del LastSGPT_cls_embedding4,
del LastSGPT_cls_embedding,
del LastSGPT_extracted_text_cls_embedding1,
del LastSGPT_extracted_text_cls_embedding2,
del LastSGPT_extracted_text_cls_embedding3,
del LastSGPT_extracted_text_cls_embedding4,
del LastSGPT_extracted_text_cls_embedding,


import gc
gc.collect()
torch.cuda.empty_cache()

## GPT 2

In [ ]:
import torch
from transformers import GPT2Model, GPT2Tokenizer

def GPT2embedding(texts):

    # Check if GPU is available and set the device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load pre-trained model and tokenizer and move the model to the GPU
    model = GPT2Model.from_pretrained('gpt2').to(device)
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))
    # Input sentences
    # texts = ['Your sentence here1', 'Your sentence here2']

    # Tokenize input sentence and move input tensors to the GPU
    input_ids = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)['input_ids'].to(device)

    # Get the hidden states (outputs from each layer)
    with torch.no_grad():
        outputs = model(input_ids)

    # # Optionally, move the result back to the CPU if needed
    # sentence_embedding = sentence_embedding.cpu().detach()

    # print(sentence_embedding.shape)
    # return sentence_embedding

    # Access last hidden states
    last_hidden_states = outputs.last_hidden_state
    print(last_hidden_states.shape)

    # 获取最后的隐藏状态 [batch_size, seq_len, hidden_size]
    last_hidden_states = outputs.last_hidden_state

    # 获取第一个 token (<s> token) 的 embedding（相当于 CLS token 的句子嵌入）
    cls_embedding = last_hidden_states[:, 0, :]

    # 也可以取所有 token 的平均值作为句子嵌入
    mean_embedding = torch.mean(last_hidden_states, dim=1)

    print(f"CLS token embedding size: {cls_embedding.shape}")
    print(f"Mean token embedding size: {mean_embedding.shape}")
    return cls_embedding,mean_embedding

In [ ]:
# GPT2_cls_embedding,GPT2_mean_embedding=GPT2embedding(text_list)

GPT2_cls_embedding1,GPT2_mean_embedding1=GPT2embedding(text_list[:1000])
GPT2_cls_embedding2,GPT2_mean_embedding2=GPT2embedding(text_list[1000:2000])
GPT2_cls_embedding3,GPT2_mean_embedding3=GPT2embedding(text_list[2000:3000])
GPT2_cls_embedding4,GPT2_mean_embedding4=GPT2embedding(text_list[3000:])

GPT2_cls_embedding=torch.cat((GPT2_cls_embedding1,GPT2_cls_embedding2,GPT2_cls_embedding3,GPT2_cls_embedding4))
GPT2_mean_embedding=torch.cat((GPT2_mean_embedding1,GPT2_mean_embedding2,GPT2_mean_embedding3,GPT2_mean_embedding4))

In [ ]:
# GPT2_extracted_text_cls_embedding,GPT2_extracted_text_mean_embedding=GPT2embedding(et_list)

GPT2_extracted_text_cls_embedding1,GPT2_extracted_text_mean_embedding1=GPT2embedding(et_list[:1000])
GPT2_extracted_text_cls_embedding2,GPT2_extracted_text_mean_embedding2=GPT2embedding(et_list[1000:2000])
GPT2_extracted_text_cls_embedding3,GPT2_extracted_text_mean_embedding3=GPT2embedding(et_list[2000:3000])
GPT2_extracted_text_cls_embedding4,GPT2_extracted_text_mean_embedding4=GPT2embedding(et_list[3000:])

GPT2_extracted_text_cls_embedding=torch.cat((GPT2_extracted_text_cls_embedding1,GPT2_extracted_text_cls_embedding2,GPT2_extracted_text_cls_embedding3,GPT2_extracted_text_cls_embedding4))
GPT2_extracted_text_mean_embedding=torch.cat((GPT2_extracted_text_mean_embedding1,GPT2_extracted_text_mean_embedding2,GPT2_extracted_text_mean_embedding3,GPT2_extracted_text_mean_embedding4))


### CKA

In [ ]:
de_biased_cka(GPT2_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(GPT2_cls_embedding[compute_list], gcn_out_embeddings),de_biased_cka(GPT2_cls_embedding[compute_list],gin_embeddings),de_biased_cka(GPT2_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(GPT2_mean_embedding[compute_list],gcn_embeddings),de_biased_cka(GPT2_mean_embedding[compute_list],gcn_out_embeddings),de_biased_cka(GPT2_mean_embedding[compute_list],gin_embeddings),de_biased_cka(GPT2_mean_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(GPT2_extracted_text_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(GPT2_extracted_text_cls_embedding[compute_list],gcn_out_embeddings),de_biased_cka(GPT2_extracted_text_cls_embedding[compute_list],gin_embeddings),de_biased_cka(GPT2_extracted_text_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(GPT2_extracted_text_mean_embedding[compute_list], gcn_embeddings),de_biased_cka(GPT2_extracted_text_mean_embedding[compute_list],gcn_out_embeddings),de_biased_cka(GPT2_extracted_text_mean_embedding[compute_list],gin_embeddings),de_biased_cka(GPT2_extracted_text_mean_embedding[compute_list],gin_out_embeddings)

In [ ]:
print(CKA(GPT2_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(GPT2_cls_embedding[compute_list], gcn_out_embeddings.to(device)),CKA(GPT2_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(GPT2_cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(GPT2_mean_embedding[compute_list],gcn_embeddings.to(device)),CKA(GPT2_mean_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(GPT2_mean_embedding[compute_list],gin_embeddings.to(device)),CKA(GPT2_mean_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(GPT2_extracted_text_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(GPT2_extracted_text_cls_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(GPT2_extracted_text_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(GPT2_extracted_text_cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(GPT2_extracted_text_mean_embedding[compute_list], gcn_embeddings.to(device)),CKA(GPT2_extracted_text_mean_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(GPT2_extracted_text_mean_embedding[compute_list],gin_embeddings.to(device)),CKA(GPT2_extracted_text_mean_embedding[compute_list],gin_out_embeddings.to(device)))

In [ ]:
del GPT2_cls_embedding1,
del GPT2_mean_embedding1,
del GPT2_cls_embedding2,
del GPT2_mean_embedding2,
del GPT2_cls_embedding3,
del GPT2_mean_embedding3,
del GPT2_cls_embedding4,
del GPT2_mean_embedding4,
del GPT2_cls_embedding,
del GPT2_mean_embedding,
del GPT2_extracted_text_cls_embedding1,
del GPT2_extracted_text_mean_embedding1,
del GPT2_extracted_text_cls_embedding2,
del GPT2_extracted_text_mean_embedding2,
del GPT2_extracted_text_cls_embedding3,
del GPT2_extracted_text_mean_embedding3,
del GPT2_extracted_text_cls_embedding4,
del GPT2_extracted_text_mean_embedding4,
del GPT2_extracted_text_cls_embedding,
del GPT2_extracted_text_mean_embedding,

import gc
gc.collect()
torch.cuda.empty_cache()

## last word GPT 2

In [ ]:
import torch
from transformers import GPT2Model, GPT2Tokenizer

def LastGPT2embedding(texts):

    # Check if GPU is available and set the device
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Load pre-trained model and tokenizer and move the model to the GPU
    model = GPT2Model.from_pretrained('gpt2').to(device)
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))
    # Input sentences
    # texts = ['Your sentence here1', 'Your sentence here2']

    # Tokenize input sentence and move input tensors to the GPU

    template = 'This_sentence_:_"The_molecule_is_a_nitrile_that_is_acetonitrile_where_one_of_the_methyl_hydrogens_is_substituted_by_a_2-methylphenyl_group."_means_in_one_word:"Acetonitrile".This_sentence_:_"*sent_0*"_means_in_one_word:"'
    inputs = tokenizer([template.replace('*sent_0*', i).replace('_', ' ') for i in texts], padding=True,  return_tensors="pt")['input_ids'].to(device)
    with torch.no_grad():
        embeddings = model(inputs, output_hidden_states=True, return_dict=True).hidden_states[-1][:, -1, :]
    print(embeddings.shape)
    return embeddings



In [ ]:
# LastGPT_cls_embedding=LastGPT2embedding(text_list)

LastGPT_cls_embedding1=LastGPT2embedding(text_list[:1000])
LastGPT_cls_embedding2=LastGPT2embedding(text_list[1000:2000])
LastGPT_cls_embedding3=LastGPT2embedding(text_list[2000:3000])
LastGPT_cls_embedding4=LastGPT2embedding(text_list[3000:])

LastGPT_cls_embedding=torch.cat((LastGPT_cls_embedding1,LastGPT_cls_embedding2,LastGPT_cls_embedding3,LastGPT_cls_embedding4))

In [ ]:
# LastGPT_extracted_text_cls_embedding=LastGPT2embedding(et_list)

LastGPT_extracted_text_cls_embedding1=LastGPT2embedding(et_list[:1000])
LastGPT_extracted_text_cls_embedding2=LastGPT2embedding(et_list[1000:2000])
LastGPT_extracted_text_cls_embedding3=LastGPT2embedding(et_list[2000:3000])
LastGPT_extracted_text_cls_embedding4=LastGPT2embedding(et_list[3000:])

LastGPT_extracted_text_cls_embedding=torch.cat((LastGPT_extracted_text_cls_embedding1,LastGPT_extracted_text_cls_embedding2,LastGPT_extracted_text_cls_embedding3,LastGPT_extracted_text_cls_embedding4))


### CKA

In [ ]:
de_biased_cka(LastGPT_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(LastGPT_cls_embedding[compute_list], gcn_out_embeddings),de_biased_cka(LastGPT_cls_embedding[compute_list],gin_embeddings),de_biased_cka(LastGPT_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(LastGPT_extracted_text_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(LastGPT_extracted_text_cls_embedding[compute_list],gcn_out_embeddings),de_biased_cka(LastGPT_extracted_text_cls_embedding[compute_list],gin_embeddings),de_biased_cka(LastGPT_extracted_text_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
print(CKA(LastGPT_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(LastGPT_cls_embedding[compute_list], gcn_out_embeddings.to(device)),CKA(LastGPT_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(LastGPT_cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(LastGPT_extracted_text_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(LastGPT_extracted_text_cls_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(LastGPT_extracted_text_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(LastGPT_extracted_text_cls_embedding[compute_list],gin_out_embeddings.to(device)))

In [ ]:
del LastGPT_cls_embedding1,
del LastGPT_cls_embedding2,
del LastGPT_cls_embedding3,
del LastGPT_cls_embedding4,
del LastGPT_cls_embedding,
del LastGPT_extracted_text_cls_embedding1,
del LastGPT_extracted_text_cls_embedding2,
del LastGPT_extracted_text_cls_embedding3,
del LastGPT_extracted_text_cls_embedding4,
del LastGPT_extracted_text_cls_embedding,

import gc
gc.collect()
torch.cuda.empty_cache()

## Summarise last word GPT 2

In [ ]:
import torch
from transformers import GPT2Model, GPT2Tokenizer

def LastSGPT2_embedding(texts):

    # Check if GPU is available and set the device
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

    # Load pre-trained model and tokenizer and move the model to the GPU
    model = GPT2Model.from_pretrained('gpt2').to(device)
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        model.resize_token_embeddings(len(tokenizer))
    # Input sentences
    # texts = ['Your sentence here1', 'Your sentence here2']

    # Tokenize input sentence and move input tensors to the GPU

    template = 'This_sentence_:_"The_molecule_is_a_nitrile_that_is_acetonitrile_where_one_of_the_methyl_hydrogens_is_substituted_by_a_2-methylphenyl_group."_summarise_in_one_word:"Acetonitrile".This_sentence_:_"*sent_0*"_summarise_in_one_word:"'
    inputs = tokenizer([template.replace('*sent_0*', i).replace('_', ' ') for i in texts], padding=True,  return_tensors="pt")['input_ids'].to(device)
    with torch.no_grad():
        embeddings = model(inputs, output_hidden_states=True, return_dict=True).hidden_states[-1][:, -1, :]
    print(embeddings.shape)
    return embeddings



In [ ]:
# LastSGPT2_cls_embedding=LastSGPT2_embedding(text_list)
LastSGPT2_cls_embedding1=LastSGPT2_embedding(text_list[:1000])
LastSGPT2_cls_embedding2=LastSGPT2_embedding(text_list[1000:2000])
LastSGPT2_cls_embedding3=LastSGPT2_embedding(text_list[2000:3000])
LastSGPT2_cls_embedding4=LastSGPT2_embedding(text_list[3000:])

LastSGPT2_cls_embedding=torch.cat((LastSGPT2_cls_embedding1,LastSGPT2_cls_embedding2,LastSGPT2_cls_embedding3,LastSGPT2_cls_embedding4))

In [ ]:
# LastSGPT2_extracted_text_cls_embedding=LastSGPT2_embedding(et_list)
LastSGPT2_extracted_text_cls_embedding1=LastSGPT2_embedding(et_list[:1000])
LastSGPT2_extracted_text_cls_embedding2=LastSGPT2_embedding(et_list[1000:2000])
LastSGPT2_extracted_text_cls_embedding3=LastSGPT2_embedding(et_list[2000:3000])
LastSGPT2_extracted_text_cls_embedding4=LastSGPT2_embedding(et_list[3000:])

LastSGPT2_extracted_text_cls_embedding=torch.cat((LastSGPT2_extracted_text_cls_embedding1,LastSGPT2_extracted_text_cls_embedding2,LastSGPT2_extracted_text_cls_embedding3,LastSGPT2_extracted_text_cls_embedding4))

### CKA

In [ ]:
de_biased_cka(LastSGPT2_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(LastSGPT2_cls_embedding[compute_list], gcn_out_embeddings),de_biased_cka(LastSGPT2_cls_embedding[compute_list],gin_embeddings),de_biased_cka(LastSGPT2_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
de_biased_cka(LastSGPT2_extracted_text_cls_embedding[compute_list], gcn_embeddings),de_biased_cka(LastSGPT2_extracted_text_cls_embedding[compute_list],gcn_out_embeddings),de_biased_cka(LastSGPT2_extracted_text_cls_embedding[compute_list],gin_embeddings),de_biased_cka(LastSGPT2_extracted_text_cls_embedding[compute_list],gin_out_embeddings)

In [ ]:
print(CKA(LastSGPT2_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(LastSGPT2_cls_embedding[compute_list], gcn_out_embeddings.to(device)),CKA(LastSGPT2_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(LastSGPT2_cls_embedding[compute_list],gin_out_embeddings.to(device)))
print(CKA(LastSGPT2_extracted_text_cls_embedding[compute_list], gcn_embeddings.to(device)),CKA(LastSGPT2_extracted_text_cls_embedding[compute_list],gcn_out_embeddings.to(device)),CKA(LastSGPT2_extracted_text_cls_embedding[compute_list],gin_embeddings.to(device)),CKA(LastSGPT2_extracted_text_cls_embedding[compute_list],gin_out_embeddings.to(device)))

In [ ]:
del LastSGPT2_cls_embedding1,
del LastSGPT2_cls_embedding2,
del LastSGPT2_cls_embedding3,
del LastSGPT2_cls_embedding4,
del LastSGPT2_cls_embedding,
del LastSGPT2_extracted_text_cls_embedding1,
del LastSGPT2_extracted_text_cls_embedding2,
del LastSGPT2_extracted_text_cls_embedding3,
del LastSGPT2_extracted_text_cls_embedding4,
del LastSGPT2_extracted_text_cls_embedding,

import gc
gc.collect()
torch.cuda.empty_cache()